## 1) Establish Stock Universe of tickers
## 2) Fetch and Store data
## 3) Screen

In [1]:
"""

Make Three Microservices:
1) Data Fetcher/Scraper/Aggregator/Manager to store/retrieve/update

2) Screener --> Strategy(Indicators)/Alerter Deployment Infrastructure

3) Aggregate Dashboard GUI & Live Graphing Application

"""

'\n\nMake Three Microservices:\n1) Data Fetcher/Scraper/Aggregator/Manager to store/retrieve/update\n\n2) Screener --> Strategy(Indicators)/Alerter Deployment Infrastructure\n\n3) Aggregate Dashboard GUI & Live Graphing Application\n\n'

In [3]:
from stockalyzer.data.data_manager import *
from stockalyzer.data.data_fetcher import TDDataFetcher, YFDataFetcher
from stockalyzer.strategy.ema_crossover import EMACrossoverStrategy
from stockalyzer.strategy.rsi_ema import RSIEMAStrategy
from stockalyzer.model.stock import Stock

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')

ModuleNotFoundError: No module named 'stockalyzer'

In [ ]:
fetcher = YFDataFetcher(['AAPL','AMZN'])
to_plot = debug = False
# data = fetcher.get_hourly_price_history(datetime(2017, 6, 1))
# fetcher = TDDataFetcher(tickers)
sm = StockDataManager.load_from_data_fetcher(fetcher, datetime(2019, 6, 1), interval='1d',debug=debug)
#     sm = StockDataManager.load_from_csv('data.csv', datetime(2019, 6, 1))
display(sm._data.head())
sm.run_strat(EMACrossoverStrategy, p1=20, p2=80, plot=to_plot)
sm.run_strat(RSIEMAStrategy, p1=9, p2=(30,70), plot=to_plot)
strat_perf = sm.get_results(to_print=True,ipython=True,graphs=to_plot)
#     stocks = [Stock(t, strategy_perf=strat_perf.loc[t], fundamentals=fundamentals.loc[t], finviz=True) for t in sm.tickers]
strat_perf

In [ ]:
sm._data['AAPL'].head()

In [ ]:
# %%time

def main(tickers=[], debug=False, to_plot=False, intervals='d'):
    fetcher = TDDataFetcher(tickers)
    fundamentals = fetcher.get_fundamentals()
    sm = StockDataManager.load_from_data_fetcher(fetcher, datetime(2017, 6, 1), debug=debug, intervals=intervals)
#     sm = StockDataManager.load_from_csv('data.csv', datetime(2019, 6, 1))
    sm.run_strat(EMACrossoverStrategy, p1=20, p2=80, plot=to_plot)
    sm.run_strat(RSIEMAStrategy, p1=9, p2=(30,70), plot=to_plot)
    strat_perf = sm.get_results(to_print=True,ipython=True,graphs=to_plot)
    stocks = [Stock(t, strategy_perf=strat_perf.loc[t], fundamentals=fundamentals.loc[t], finviz=True) for t in sm.tickers]
    return stocks, sm
# start = time.time()
tickers = ["NVDA", "AMD"]
stocks, sm = main(tickers=tickers, to_plot=True)
# end = time.time()
# print('{0:.4f} seconds'.format(end - start))

In [ ]:



# """YAHOO FINANCE"""
# import yfinance as yf
# # YAHOO FINANCEDATA
# tickers = ['AAPL','NVDA']
# yf_tickers = [yf.Ticker(t) for t in tickers]
# data = yf.download(tickers=tickers, start=datetime(2010, 6, 1), interval='1d', group_by = 'ticker')
# data['AAPL']

# TODO
## Cibelli:
- research successful strategy
## Me:
- Yahoo Fin Split & Adj Close data
- Hourly data
- Test capacity of algo to see how many tickers can run in time period (ideal 30 - 60 mins)
- Perf:
        - Time per trade
        - 

# TODO:
### - optstrategy --> more params, grid search
### Fix load_from_csv start_time param
## Data:
- Split & Dividend Data

# NEXT:
### - Screener Infrastructure / Stock Universe

# AI/ML:
### - create model to aggregate all info for a given stock, such as fundamentals, strategy performance, news...
### - create NN using ^features^ 

In [ ]:
def get_perf(results: List[List[bt.cerebro.OptReturn]])->pd.DataFrame:
    stats = []
    for i in results:
        for j in i:
            stats.append(
                {'strategy': f'SMA_Cross_{j.params.fast}_{j.params.slow}',
                 'sharpe_ratio': j.analyzers.sharperatio.get_analysis()['sharperatio'],
                 'return': '{0:.2f}%'.format(list(j.analyzers.timereturn.get_analysis().values())[0] * 100),
                 'buy_and_hold': '{0:.2f}%'.format(list(j.analyzers.buyandhold.get_analysis().values())[0] * 100),
                 'vwr': j.analyzers.vwr.get_analysis()['vwr'],
                }
            )
    df = pd.DataFrame(stats)
    df.sort_values(by='sharpe_ratio', ascending=False, inplace=True)
    df.set_index('strategy', inplace=True)
    return df

def grid_search(data_feed: bt.feeds.PandasData, fast: List[int], slow: List[int])->List[List[bt.cerebro.OptReturn]]:
    cerebro = bt.Cerebro()
    cerebro.optstrategy(EMACrossoverStrategy, fast=fast, slow=slow)
    cerebro.adddata(data_feed)

    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(commission=0.0004)  # Binance Swap taker fee

    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, factor=365)
    cerebro.addanalyzer(bt.analyzers.VWR, timeframe=bt.TimeFrame.Days, tann=365)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame, data=data_feed, _name='buyandhold')

    return cerebro.run()

# data_feed = bt.feeds.PandasData(dataname=sm._data.loc['PYPL'], timeframe=bt.TimeFrame.Days, openinterest=None)
# results = grid_search(data_feed, fast=[32, 64], slow=[256, 512])

In [ ]:
# # General STOCK SCREENER:
# output_file = "screener_results.csv"
# result = subprocess.run([sys.executable, "finviz_screener.py", "--table=Valuation",f"--output_name={output_file.split('.')[0]}", "--save=True"], capture_output=True, text=True)
# pd.read_csv(output_file)

In [ ]:
import yfinance as yf
tickers = ['AAPL','MSFT','AMZN','GOOG']

for t in tickers:
    data = yf.Ticker("AAPL")
    aapl.history(period='max')

In [ ]:
portfolio.splits

In [ ]:
aapl.actions

In [ ]:
aapl.info